In [ ]:
#This program was used to assist for manual labelling of tweets

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import random
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

In [ ]:
df=pd.read_csv('../Data/cleanTweets.csv',na_values={'None','NONE'},nrows=15000)

df=df.replace(r'^\s*$', np.nan, regex=True)
df=df.fillna('')

try:
    df_label=pd.read_csv('../Data/labelledTweets.csv')
except:
    df_label=pd.DataFrame()

count=0
with open('status.txt','r') as f: #status of count of labelled tweets and index of next tweet to be labelled is saved in 
                                  #secondary storage
    
    ind,count=f.read().split('!##!') # "!##!" is being used as delimiter between count and index 
    ind=int(ind)
    count=int(count)
    print(ind,' ',count)

In [ ]:
import warnings
warnings.simplefilter('ignore')

obj_vad=SIA() #"VADER" lexical anayzer is being used for assistance in labelling
              #Although, here VADER only suggests a label for given tweet, ultimate decision is in the hands
              #of person labelling tweets (person may accept a label suggested by VADER or may give some other 
              #label he/she deems suitable)
prev=ind
start=ind
end=15000
   
for i in np.arange(start,end):
    if df.loc[i,:]['full_text']=='':
        s=df.loc[i,:]['text']
    else:
        s=df.loc[i,:]['full_text']
    
    pred=obj_vad.polarity_scores(text=s)
    if pred['compound'] > 0:
        lab='pos'
    if pred['compound'] == 0:
        lab='neu'  
    if pred['compound'] < 0:
        lab='neg'
    
    lab=pred[0]
    print(i,' ',count,'\n',lab,'\n', s)

    try:
            opt=input()
            if opt=='s': #save command
                print('Last saved till index: ',prev,'and count: ',count )
                break
            else:
                if opt=='8':#to accept label suggested by VADER 
                    df_label=df_label.append(pd.DataFrame(data={'text':[s],'labels':[lab]}),sort=True )
                    count=count+1
                else:#if VADER's label is incorrect, assign correct one

                    if opt=='6': #assign positive label
                        df_label=df_label.append(pd.DataFrame(data={'text':[s],'labels':['pos']}),sort=True )
                        count=count+1
                    if opt=='4': #assign negative label
                        df_label=df_label.append(pd.DataFrame(data={'text':[s],'labels':['neg']}),sort=True )
                        count=count+1
                    if opt=='5': #assign neutral label
                        df_label=df_label.append(pd.DataFrame(data={'text':[s],'labels':['neu']}),sort=True )
                        count=count+1
                       

    except Exception as e:

            print(e)
            raise
            
    prev=i

    
df_label.to_csv('../Data/labelledTweets.csv',index=None,header=True)    

with open('status.txt','w') as f:    
    f.write(str(prev+1) + '!##!'+ str(count))